In [1]:
#https://plot.ly/python/geo-streaming/
import plotly

In [2]:
import plotly.plotly as py  
import plotly.tools as tls   
import plotly.graph_objs as go
import numpy as np 

In [3]:
import pandas as pd
dframe = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')

In [4]:
stream_tokens = tls.get_credentials_file()['stream_ids']
token = stream_tokens[-2]   # I'm getting my stream tokens from the end to ensure I'm not reusing tokens
# print token


In [5]:
stream_id = dict(token=token, maxpoints=3)

In [6]:
data = [dict(
        type='scattergeo',
        lon=[],
        lat=[],
        mode='markers',
        marker=dict(
            size=8,
            opacity=0.8,
            reversescale=True,
            autocolorscale=False,
            line=dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
        ),
        stream=stream_id,
        name="Plane")]

layout = dict(
        title = 'Busy Airplane Streaming',
        colorbar = False,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='geo-streaming2', auto_open=False, fileopt='extend')

Now let's set up a stream link object for our geo-scatter trace and start streaming some data to our plot

In [7]:
s = py.Stream(stream_id=token)

# Start Streaming

In [ ]:
import datetime
import time

def lets_stream():

    s.open()

    airports = dframe.sample(4)[['lat', 'long', 'airport']]
    depart = airports.iloc[0]
    arrive = airports.iloc[1]
    num_steps = 20

    while True:

        count = 0

        lats = np.linspace(depart['lat'], arrive['lat'], num_steps)
        lons = np.linspace(depart['long'], arrive['long'], num_steps)

        for i, j in zip(lats, lons):

            # added pts for the departure and arrival airports!!!
            s.write(dict(lon=[depart['long'], j, arrive['long']],
                         lat=[
                             depart['lat'], i, arrive['lat']], type='scattergeo',
                         marker={'size': [5, 7 + 0.2 * count, 5], 'sizemode': 'area',
                                 'color': ["green", "blue", "red"],
                                 'symbol': ["circle", "star", "x-open"]},
                         text=[depart['airport'],
                               '{},{}'.format(count, datetime.datetime.now()),
                               arrive['airport']]))

            count += 1
            stall = np.random.normal(10, 3)
            time.sleep(int((abs(stall) + 0.01) / 2.0))
            s.heartbeat()
            time.sleep(int((abs(stall) + 0.01) / 2.0))

        depart = arrive
        arrive = dframe.sample(1)[['lat', 'long', 'airport']].iloc[0]

while True:
    try:
        lets_stream()
    except Exception as e:
        with open('log.txt', 'a+') as f:
            f.write(str(e))
        print(str(e))
        s.close()

In [11]:
tls.embed('streaming-demos','121')